<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/image_gen/gemini_try_failed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install python-dotenv pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 6.5 MB/s eta 0:00:00


In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
# Start Ollama server

import subprocess

# Start the command in a background process
process = subprocess.Popen(["ollama", "serve"])

# The kernel can continue execution while the process runs in the background
print("The 'ollama serve' process is running in the background.")

The 'ollama serve' process is running in the background.


In [4]:
#!ollama pull llama3.2:1b

In [5]:
# install the Ollama python API package

!pip install ollama

In [36]:

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
if GOOGLE_API_KEY :
  print("GOOGLE_API_KEY is set")

GOOGLE_API_KEY is set


In [6]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [10]:
! pip install crewai-tools[mcp]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.6/324.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.9 MB/s e

In [11]:
from crewai_tools import DallETool

In [12]:
from crewai import Agent, LLM

In [15]:
gemini = LLM(
    api_key=os.getenv("GEMINI_API_KEY"),
    model="gemini/gemini-2.5-flash",
)
my_llm=gemini

In [17]:
story_teller = Agent(
    role = 'pro content creator for youtube shorts',
    goal = '''Take the high level topic described here and prepare a very intresting and very engrosing story for a youtube video.
    Story should be narrated within four minutes. strory should have intresting start and amzing end and should ensure listener is hooked throught story.
    The python module must be completely self-contained, and ready so that it can be tested or have a simple UI built for it.
    here is the topic: {topic}''',
    backstory = '''You're a seasoned a seasoned story teller reknowned for compelling story telling''',
    llm = my_llm,
    verbose=True,
)

In [18]:
photographer = Agent(
    role = 'you create very compelling photo with great detail',
    goal = ''' you create photos based on story told by story teller and topic. you take for every 2 sentences in story, you create one photo which describe the scene in great detail. also create photo with your tool''',
    backstory = '''You're a seasoned photo creater with a knack for creating amazingly attractive photos''',
    llm = my_llm,
    tools=[ DallETool()],
    context=[story_teller],
    verbose=True,
)

In [20]:

import os

from crewai import Agent, Task, Crew, Process, LLM

In [22]:
# Create tasks for your agents
story_teller_task = Task(
  description="""Take the given topic {topic} and write a captivating YouTube video story that can be narrated in four minutes. The story must have an intriguing start, an engaging middle, and an amazing end. Structure the story into three labeled parts: 'Beginning', 'Middle', and 'End'. Also, provide a catchy title at the top. The entire output must be in plain text without any markdown""",
  agent=story_teller,
  verbose=True,
  expected_output="""Expected Output:

A string in the following format:

"Title: [Title of the story]

Beginning: [Text for the beginning]

Middle: [Text for the middle]

End: [Text for the end]"

The story should be approximately 500-600 words in total."""

)

In [26]:
# Create tasks for your agents
photographer_task = Task(
  description="""give the {topic} and story given by story teller in context, create a photo using tools""",
  agent=photographer,
  verbose=True,
  expected_output="a photos"

)

In [27]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[story_teller,photographer],
  tasks=[story_teller_task,photographer_task],
  verbose=False, # You can set it to 1 or 2 to different logging levels
)


In [28]:
crew

Crew(id=e46f078b-aa16-457a-9940-5c1c2ff8aded, process=Process.sequential, number_of_agents=2, number_of_tasks=2)

In [29]:
topic = """ indian historical stories
"""

template_input = {"topic": topic}

In [37]:
# Get your crew to work!
#result = crew.kickoff()
result = crew.kickoff(inputs=template_input)

# Agent: pro content creator for youtube shorts
## Task: Take the given topic  indian historical stories
 and write a captivating YouTube video story that can be narrated in four minutes. The story must have an intriguing start, an engaging middle, and an amazing end. Structure the story into three labeled parts: 'Beginning', 'Middle', and 'End'. Also, provide a catchy title at the top. The entire output must be in plain text without any markdown


# Agent: pro content creator for youtube shorts
## Final Answer: 
Title: The Emperor Who Forged Peace From Bloodshed: Ashoka's Transformation

Beginning:
Imagine an emperor, driven by an unquenchable thirst for power, his name echoing across ancient India not as a unifier, but as ‘Chandashoka’ – Ashoka the Fierce. His empire, the mighty Mauryan, stretched vast, yet one independent kingdom remained, a thorn in his side: Kalinga. It was a prosperous land, strategically vital, and a symbol of defiance. For years, whispers of its unconquered spi

In [ ]:
gemini_flash="gemini-2.0-flash"
gemma_1b="gemma-3-1b-it"
gemma_4b="gemma-3-4b-it"
gemma_12b="gemma-3-12b-it"
gemma_27b="gemma-3-27b-it"
gemma_e4b="gemma-3n-e4b-it"
llama3="llama3.2:1b"
mistral="mistral:7b-instruct"


In [ ]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [ ]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [34]:
from openai import OpenAI

In [38]:
client = OpenAI()

PROMPT = "A vaporwave computer"

In [39]:
response = client.images.generate(
    model="dall-e-3",
    prompt=PROMPT,
)

BadRequestError: Error code: 400 - {'error': {'message': None, 'type': 'image_generation_user_error', 'param': None, 'code': None}}

In [53]:
!sudo apt-get install imagemagick

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libdjvulibre-text libdjvulibre21
  libfftw3-double3 libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0
  libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10
  libwmflite-0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng enscript gimp
  gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance sane-utils
  texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev inkscape
  poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fon

In [40]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64

In [41]:
client = genai.Client()

contents = ('Hi, can you create a 3d rendered image of a pig '
            'with wings and a top hat flying over a happy '
            'futuristic scifi city with lots of greenery?')

In [42]:
response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents=contents,
    config=types.GenerateContentConfig(
      response_modalities=['TEXT', 'IMAGE']
    )
)

In [45]:
for part in response.candidates[0].content.parts:
  if part.text is not None:
    print(part.text)

  if part.inline_data is not None:
    image = Image.open(BytesIO((part.inline_data.data)))
    image.save('gemini-native-image.png')
    image.show()

I will create a 3D rendered image of a whimsical scene. Imagine a pink pig with a neatly perched grey top hat and feathery white wings soaring through the bright blue sky. Below, a vibrant futuristic cityscape unfolds, characterized by sleek, rounded buildings in pastel shades and interwoven with lush green parks and elevated gardens. The overall atmosphere should be cheerful and optimistic, with a sense of playful wonder.




In [46]:
!ls -ltr

total 1264
drwxr-xr-x 1 root root    4096 Jun 18 13:35 sample_data
-rw-r--r-- 1 root root 1290095 Jun 23 12:27 gemini-native-image.png


In [54]:
image = Image.open(BytesIO((part.inline_data.data)))
image.save('gemini-native-image.png')
image.show()

In [55]:
from IPython.display import Image
Image(filename='gemini-native-image.png'')

SyntaxError: unterminated string literal (detected at line 2) (ipython-input-55-981193524.py, line 2)

In [60]:
MODEL_ID="gemini-2.0-flash-preview-image-generation"

In [62]:
from IPython.display import Image, Markdown, display
from google import genai
from google.genai.types import GenerateContentConfig, Part

In [63]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Create a tutorial explaining how to make a peanut butter and jelly sandwich in three easy steps. For each step, provide a title with the number of the step, an explanation, and also generate an image, generate each image in a 1:1 aspect ratio.",
    config=GenerateContentConfig(
        response_modalities=["TEXT", "IMAGE"],
        safety_settings=[
            {"method": "PROBABILITY"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT"},
            {"threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ],
    ),
)

ValueError: method parameter is not supported in Gemini API.